<a href="https://colab.research.google.com/github/CharlotteY2003/MIT-Futuremakers/blob/main/7_29_Computer_Vision.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [27]:
import keras
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import *
from keras.callbacks import *
from keras.optimizers import *
import os
from keras.preprocessing.image import img_to_array
from keras.preprocessing import image
import cv2
import numpy as np

In [19]:
num_classes = 6
width,height = 48,48
batch_size = 32

In [22]:
train_data_dir = './drive/MyDrive/train'
val_data_dir = 'drive/MyDrive/validation'

In [21]:
os.listdir('drive/MyDrive/train')

['.DS_Store', 'Happy', 'Neutral', 'Surprise', 'Sad', 'Angry', 'Fear']

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [23]:
train_data_gen = ImageDataGenerator(
    rescale = 1.0/255,
    rotation_range = 30,
    shear_range = .3,
    zoom_range = .3,
    width_shift_range = .4,
    height_shift_range = .4,
    horizontal_flip = True,
    fill_mode = 'nearest'
)
validation_data_gen = ImageDataGenerator(rescale=1.0/255)

In [24]:
train_generator = train_data_gen.flow_from_directory(train_data_dir, target_size=(width,height),color_mode='grayscale')
validation_generator = validation_data_gen.flow_from_directory(val_data_dir,target_size=(width,height), color_mode='grayscale')

Found 28353 images belonging to 6 classes.
Found 3534 images belonging to 6 classes.


In [26]:
model = Sequential()

model.add(Conv2D(32,(3,3),padding='same',kernel_initializer='he_normal',
                 input_shape=(width,height,1)))
model.add(Activation('elu'))
model.add(BatchNormalization())
model.add(Conv2D(32,(3,3),padding='same',kernel_initializer='he_normal',
                 input_shape=(width,height,1))) #Why do you need another input shape here?
model.add(Activation('elu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.2))

model.add(Conv2D(64,(3,3),padding='same',kernel_initializer='he_normal'))
model.add(Activation('elu'))
model.add(BatchNormalization())
model.add(Conv2D(64,(3,3),padding='same',kernel_initializer='he_normal'))
model.add(Activation('elu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.2))

model.add(Conv2D(128,(3,3),padding='same',kernel_initializer='he_normal'))
model.add(Activation('elu'))
model.add(BatchNormalization())
model.add(Conv2D(128,(3,3),padding='same',kernel_initializer='he_normal'))
model.add(Activation('elu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.2))

model.add(Conv2D(256,(3,3),padding='same',kernel_initializer='he_normal'))
model.add(Activation('elu'))
model.add(BatchNormalization())
model.add(Conv2D(256,(3,3),padding='same',kernel_initializer='he_normal'))
model.add(Activation('elu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.2))

model.add(Flatten())
model.add(Dense(64,kernel_initializer='he_normal'))
model.add(Activation('elu'))
model.add(BatchNormalization())
model.add(Dropout(0.5))

model.add(Dense(64,kernel_initializer='he_normal'))
model.add(Activation('elu'))
model.add(BatchNormalization())
model.add(Dropout(0.5))

model.add(Dense(num_classes,kernel_initializer='he_normal'))
model.add(Activation('softmax'))

model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 48, 48, 32)        320       
_________________________________________________________________
activation (Activation)      (None, 48, 48, 32)        0         
_________________________________________________________________
batch_normalization (BatchNo (None, 48, 48, 32)        128       
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 48, 48, 32)        9248      
_________________________________________________________________
activation_1 (Activation)    (None, 48, 48, 32)        0         
_________________________________________________________________
batch_normalization_1 (Batch (None, 48, 48, 32)        128       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 24, 24, 32)       

In [28]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [29]:
checkpoint = ModelCheckpoint('model.h5', verbose=1,mode='min', save_best_only=True)
es = EarlyStopping(min_delta=0, patience=3,restore_best_weights=True,verbose=1)
rdl = ReduceLROnPlateau(factor=.2,patience=3,min_delta=.0001,verbose=1)
callbacks = [checkpoint,es,rdl]

In [ ]:
train_samps = 28353
val_samps = 3534
history = model.fit_generator(train_generator, steps_per_epoch=train_samps//batch_size,epochs=25,
                              validation_data=validation_generator,validation_steps=val_samps//batch_size)

/usr/local/lib/python3.7/dist-packages/keras/engine/training.py:1915: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/25
 81/886 [=>............................] - ETA: 4:40:33 - loss: 2.9472 - accuracy: 0.1712

In [ ]:
face_classifier = cv2.CascadeClassifier('/haarcascade_frontalface_default.xml')
classifier = load_model('model.h5')

In [ ]:
class_labels = ['Angry','Fear','Happy','Neutral','Sad','Surprise']
cap = cv2.VideoCapture(0) #0=use primary webcam of device

In [ ]:
#Boxes around faces
while True:
  ret,frame = cap.read() #Runs until 'q' is pressed on keyboard

  gray = cv2.cvtColor(frame,cv2.COLORBGR2GRAY) #converts image to grayscale
  faces = face_classifier.detectMultiScale(gray,1.3,5) #detects and returns coordinates of faces

  for (x,y,w,h) in faces: #create rectangle
    cv2.rectangle(frame, (x,y),(x+w,y_h),(255,0,0),2)
    roi_gray = gray[y:y+h,x:x+w]
    roi_gray = cv2.resize(roi_gray,(48,48), interpolation=cv2.INTER_AREA)

    if np.sum([roi_gray] !=0):
      roi = roi_gray.astype('float32')/255.0
      roi = img_to_array(roi)
      roi = np.expand_dims(roi,axis=0)
      y_pred = classifier.predict(roi)[0]
      label = class_labels[y_pred.argmax()]
      label_position = (x,y)
      cv2.putText(frame,label,label_position,cv2.FONT_HERSHEY_SIMPLEX,2, (0,255,0),3)
    else:
      cv2.putText(frame,'No face found', (20,20),cv2.FONT_HERSHEY_SIMPLEX,2, (0,255,0),3)

  cv2.imshow('Emotion Detector', frame)
  if cv2.waitKey(1) & 0xFF==ord('q'):
    break
cap.release()
cv2.destroyAllWindows() 